## Exemple de base de donnée avec des connexions many to many

### Configuration et création

In [71]:
rm app.db

In [72]:
# %%file config.py
# Sera utilisé pour configurer la db
import os
basedir  = os.getcwd()
basedir = basedir[0] if isinstance(basedir, list) else basedir

class Config(object):
    # ...
    SQLALCHEMY_DATABASE_URI = os.environ.get('DATABASE_URL') or \
        'sqlite:///' + os.path.join(basedir, 'app.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = False

In [73]:
#https://docs.sqlalchemy.org/en/14/orm/basic_relationships.html#many-to-many
#from sqlalchemy import Table, Column, Integer, ForeignKey
#from sqlalchemy.orm import relationship
#from sqlalchemy.ext.declarative import declarative_base

from flask import Flask
#from config import Config # si la config est définie dans config.py
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
db = SQLAlchemy(app)
Base = db.Model 


association_table = db.Table('association', Base.metadata,
    db.Column('parents', db.Integer, db.ForeignKey('parents.id')),
    db.Column('children', db.Integer, db.ForeignKey('children.id'))
)


class Parent(Base):
    __tablename__ = 'parents'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64))
    children = db.relationship(
        "Child",
        secondary=association_table,
        back_populates="parents")

class Child(Base):
    __tablename__ = 'children'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64))
    parents = db.relationship(
        "Parent",
        secondary=association_table,
        back_populates="children")
    
    
    
db.create_all() # Création des tables    


/home/bercherj/.local/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:812: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
/home/bercherj/.local/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


### Exemples d'utilisation, queries

In [74]:
u = Parent(name='john' )
v = Parent(name='kelly')
r = Child(name='bob', parents=[u,v])
t = Child(name='john', parents=[v,u])

In [75]:
db.session.add(u)
db.session.add(v)
db.session.commit()
db.session.add(r)
db.session.add(t)
db.session.commit()


In [76]:
r.parents

[<Parent 1>, <Parent 2>]

In [77]:
for u in v.children:
    print(u.name)
    

bob
john


In [78]:
Child.query.all()

[<Child 1>, <Child 2>]

In [79]:
Child.query.filter(Parent.name=='john').all()

[<Child 1>, <Child 2>]

In [80]:
Child.query.filter(Parent.name=='john').first()

<Child 1>

## Base many to many pour le TP BibApp

In [81]:
rm app.db

rm: cannot remove 'app.db': No such file or directory


In [82]:
# %%file config.py
# Sera utilisé pour configurer la db
import os
basedir  = os.getcwd()
basedir = basedir[0] if isinstance(basedir, list) else basedir

class Config(object):
    # ...
    SQLALCHEMY_DATABASE_URI = os.environ.get('DATABASE_URL') or \
        'sqlite:///' + os.path.join(basedir, 'app.db')
    SQLALCHEMY_TRACK_MODIFICATIONS = False

In [83]:
#https://docs.sqlalchemy.org/en/14/orm/basic_relationships.html#many-to-many
#from sqlalchemy import Table, Column, Integer, ForeignKey
#from sqlalchemy.orm import relationship
#from sqlalchemy.ext.declarative import declarative_base

from flask import Flask
#from config import Config # si la config est définie dans config.py
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config.from_object(Config)
db = SQLAlchemy(app)
Base = db.Model 


association_table = db.Table('association', Base.metadata,
    db.Column('auteurs', db.Integer, db.ForeignKey('authors.id')),
    db.Column('publis', db.Integer, db.ForeignKey('publis.id')),
    db.Column('labos', db.Integer, db.ForeignKey('labos.id'))                             
)



class Labo(db.Model):
    __tablename__ = 'labos'
    id = db.Column(db.Integer, primary_key=True)
    labname = db.Column(db.String(64), index=True)
    membres = db.relationship('ESIEEAuthor', backref='labo', lazy='dynamic') # crée un lien avec un attribut "author" pour chacun des posts
    publis = db.relationship(
        "Publis",
        secondary=association_table,
        back_populates="labos")    

    def __repr__(self):
        return '<Lab {}>'.format(self.labname)

class ESIEEAuthor(Base):
    __tablename__ = 'authors'
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(64))
    prenom = db.Column(db.String(64))
    HAL = db.Column(db.String(64))
    debut_activite = db.Column(db.Integer)
    fin_activite = db.Column(db.Integer)
    publis = db.relationship(
        "Publis",
        secondary=association_table,
        back_populates="Author")
    labo_id = db.Column(db.Integer, db.ForeignKey('labos.id'))

    def __repr__(self):
        return '<Author {}>'.format(self.nom)

"""
association_table = db.Table('association', Base.metadata,
    db.Column('auteurs', db.Integer, db.ForeignKey('authors.id')),
    db.Column('publis', db.Integer, db.ForeignKey('publis.id')),
    db.Column('labos', db.Integer, db.ForeignKey('labos.id'))                             
)



class Labo(db.Model):
    __tablename__ = 'labos'
    id = db.Column(db.Integer, primary_key=True)
    labname = db.Column(db.String(64), index=True)
    membres = db.relationship('ESIEEAuthor', backref='labo', lazy='dynamic') # crée un lien avec un attribut "author" pour chacun des posts
    publis = db.relationship(
        "Publis",
        secondary=association_table,
        back_populates="labos")    

    def __repr__(self):
        return '<Lab {}>'.format(self.labname)

class ESIEEAuthor(Base):
    __tablename__ = 'authors'
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(64))
    prenom = db.Column(db.String(64))
    HAL = db.Column(db.String(64))
    debut_activite = db.Column(db.Integer)
    fin_activite = db.Column(db.Integer)
    publis = db.relationship(
        "Publis",
        secondary=association_table,
        back_populates="authors")
    labo_id = db.Column(db.Integer, db.ForeignKey('labos.id'))

    def __repr__(self):
        return '<Author {}>'.format(self.nom)


class Publis(Base):
    __tablename__ = 'publis'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64))
    authors = db.relationship(
        "ESIEEAuthor",
        secondary=association_table,
        back_populates="publis")
    labos = db.relationship(
        "Labo",
        secondary=association_table,
        back_populates="publis")    

    def __repr__(self):
        return '<Publis {}>'.format(self.name)    
"""    

class Publis(Base):
    __tablename__ = 'publis'
    id = db.Column(db.Integer, primary_key=True)
    #id = db.Column(db.String(64), primary_key=True)
    BibliographyType = db.Column(db.String(64))
    authors = db.Column(db.String(128))    
    ISBN = db.Column(db.String(64))
    Identifier = db.Column(db.String(64))
    Title = db.Column(db.String(64))
    Journal = db.Column(db.String(64))
    Volume = db.Column(db.String(64))
    Number = db.Column(db.String(64))
    Month = db.Column(db.String(64))
    Pages = db.Column(db.String(64))
    Year = db.Column(db.Integer)
    Address = db.Column(db.String(64))
    Note = db.Column(db.String(64))
    URL = db.Column(db.String(64))
    Booktitle = db.Column(db.String(64))
    Chapter = db.Column(db.String(64))
    Edition = db.Column(db.String(64))
    Series = db.Column(db.String(64))
    Editor = db.Column(db.String(64))
    Publisher = db.Column(db.String(64))
    ReportType = db.Column(db.String(64))
    Howpublished = db.Column(db.String(64))
    Institution = db.Column(db.String(64))
    Organizations = db.Column(db.String(64))
    School = db.Column(db.String(64))
    Annote = db.Column(db.String(64))
    Custom1 = db.Column(db.String(64))
    Custom2 = db.Column(db.String(64))
    Custom3 = db.Column(db.String(64))
    Custom4 = db.Column(db.String(64))
    Custom5 = db.Column(db.String(64))
    Author = db.relationship(
        "ESIEEAuthor",
        secondary=association_table,
        back_populates="publis")
    labos = db.relationship(
        "Labo",
        secondary=association_table,
        back_populates="publis")    

    def __repr__(self):
        return '<Publis {}>'.format(self.Identifier)    

"""
'BibliographyType', 'ISBN', 'Identifier', 'Author', 'Title', 'Journal',
       'Volume', 'Number', 'Month', 'Pages', 'Year', 'Address', 'Note', 'URL',
       'Booktitle', 'Chapter', 'Edition', 'Series', 'Editor', 'Publisher',
       'ReportType', 'Howpublished', 'Institution', 'Organizations', 'School',
       'Annote', 'Custom1', 'Custom2', 'Custom3', 'Custom4', 'Custom5'
"""
        
db.create_all() # Création des tables    

### Exemple d'utilisation

In [84]:
lab1 = Labo(labname='esycom' )
lab2 = Labo(labname='ligm' )
a1 = ESIEEAuthor(nom='Bill', labo=lab2)
a2 = ESIEEAuthor(nom='Myriam', labo=lab1)
a3 = ESIEEAuthor(nom='Joe', labo=lab2)
a4 = ESIEEAuthor(nom='kelly', labo=lab1)
p1 = Publis(Identifier='pub1', Author=[a1,a2,a3], labos=[lab1])
p2 = Publis(Identifier='pub2', Author=[a2,a4], labos=[lab1, lab2])
p3 = Publis(Identifier='pub3', Author=[a2])
p4 = Publis(Identifier='pub4', Author=[a3])
p5 = Publis(Identifier='pub5', Author=[a2,a3])

In [85]:
db.session.add(lab1)
db.session.add(lab2)
db.session.commit()
db.session.add(a1)
db.session.add(a2)
db.session.commit()
db.session.add(p1)
db.session.add(p2)
db.session.commit()

In [86]:
db.session.add(p3)
db.session.add(p4)
db.session.add(p5)
db.session.commit()

In [87]:
p12 = Publis(Identifier='pub5', Author=[a2,a3], Year=44)

In [88]:
type(p12.Year)

int

In [89]:
# Même id, cela écrase la référence précédente
#p7 = Publis(id='pub1', Author=[a1,a2,a3], labos=[lab1])
#db.session.commit()

In [90]:
Publis.query.filter(Publis.id=='pub72').first()

In [91]:
# Toutes les publis
p = Publis.query.all()
p

[<Publis pub1>,
 <Publis pub4>,
 <Publis pub2>,
 <Publis pub3>,
 <Publis pub5>,
 <Publis pub5>]

In [92]:
# Liste des labos
labs = Labo.query.all()
labs

[<Lab esycom>, <Lab ligm>]

In [93]:
# Liste des publis pour un labo
ll = Publis.query.join(Publis.Author).join(ESIEEAuthor.labo).filter(Labo.labname=="ligm").all() 
ll

[<Publis pub4>, <Publis pub5>, <Publis pub1>, <Publis pub5>]

In [94]:
Publis.query.all()

[<Publis pub1>,
 <Publis pub4>,
 <Publis pub2>,
 <Publis pub3>,
 <Publis pub5>,
 <Publis pub5>]

In [95]:
# Liste des auteurs pour un labo
Labo.query.filter(Labo.labname=='ligm').first().membres.filter().all()

[<Author Bill>, <Author Joe>]

In [96]:
ESIEEAuthor.query.filter().all()

[<Author Myriam>, <Author Bill>, <Author Joe>, <Author kelly>]

In [97]:
p[0].Author

[<Author Bill>, <Author Myriam>, <Author Joe>]

In [98]:
for auteur in p[0].Author:
    print(auteur.labo)

<Lab ligm>
<Lab esycom>
<Lab ligm>


In [99]:
ll = Labo.query.all()
ll 

[<Lab esycom>, <Lab ligm>]

In [100]:
ll[1].membres

In [101]:
for uu in ll[0].membres:
    print(uu)

<Author Myriam>
<Author kelly>


In [102]:
ll[0].publis

[<Publis pub1>, <Publis pub2>]

### Initialisation de la base avec les labos

In [103]:
labnames = ['LIGM', 'ESYCOM', 'LISIS', 'IRG', 'ESIEE']
labDict = {}
for lab in labnames:
    l = Labo(labname=lab)
    labDict[lab] = l 
    db.session.add(l)
    db.session.commit()

In [104]:
Labo.query.all()

[<Lab esycom>,
 <Lab ligm>,
 <Lab LIGM>,
 <Lab ESYCOM>,
 <Lab LISIS>,
 <Lab IRG>,
 <Lab ESIEE>]

In [105]:
labDict

{'LIGM': <Lab LIGM>,
 'ESYCOM': <Lab ESYCOM>,
 'LISIS': <Lab LISIS>,
 'IRG': <Lab IRG>,
 'ESIEE': <Lab ESIEE>}

In [106]:
cd /home/bercherj/JFB/bibapp 

/home/bercherj/JFB/bibapp


### Initialisation de la base avec les auteurs

In [107]:
"""from app import db
from app.models import ESIEEAuthor, Labos

import csv
import re"""

import pandas as pd

Tab = pd.read_excel("listeBiblio.xls")



for idx in Tab.index:
    try:
        deb = int(Tab.loc[idx, 'Début'])
    except:
        deb=0
    try:
        fin = int(Tab.loc[idx, 'Fin'])
    except:
        fin=2999    
        
    auteur = ESIEEAuthor(
        nom=Tab.loc[idx, 'Nom'],
        prenom=Tab.loc[idx, 'Prenom'],
        labo=labDict.get(Tab.loc[idx, 'Labo'], labDict['ESIEE']),
        HAL=Tab.loc[idx, 'HAL'],
        debut_activite=deb,
        fin_activite=fin
    )
    db.session.add(auteur)
    db.session.commit()
    

In [108]:
labDict.get(Tab.loc[idx, 'Labo'], labDict['ESIEE'])

<Lab ESIEE>

In [109]:
Tab.columns

Index(['NomPrenom', 'Nom', 'Prenom', 'Labo', 'HAL', 'Début', 'Fin'], dtype='object')

In [110]:
ESIEEAuthor.query.all()

[<Author Myriam>,
 <Author Bill>,
 <Author Joe>,
 <Author kelly>,
 <Author BERTRAND>,
 <Author MAINWARING>,
 <Author BAUDOIN>,
 <Author DE CAMBRY>,
 <Author FAKRI>,
 <Author COUPRIE>,
 <Author SERRANO>,
 <Author VILLEGAS>,
 <Author ANDERSON>,
 <Author DHERMIES>,
 <Author GEORGES>,
 <Author LATORRE>,
 <Author DEGNY>,
 <Author KAHANE>,
 <Author REAMA>,
 <Author CELA>,
 <Author LAHILLE>,
 <Author NATOWICZ>,
 <Author ROUDIER>,
 <Author DOUAY>,
 <Author SEVELY>,
 <Author MIMOUN>,
 <Author REILLE>,
 <Author VENARD>,
 <Author WIRTH>,
 <Author BUREAU>,
 <Author DEBBASCH>,
 <Author LATORRE>,
 <Author LEFEBVRE>,
 <Author SCHOEN>,
 <Author EVE>,
 <Author BLANCHARD>,
 <Author COURIVAUD>,
 <Author CHU>,
 <Author BERCHER>,
 <Author DELABIE>,
 <Author PERROTON>,
 <Author POINTET>,
 <Author POULICHET>,
 <Author TALBOT>,
 <Author TODMAN>,
 <Author BERLAND>,
 <Author GEORGE>,
 <Author NAJMAN>,
 <Author BOUROUINA>,
 <Author ZAMPERA>,
 <Author MADAOUI>,
 <Author REDIS>,
 <Author HABIB>,
 <Author LISSORGUE

In [111]:
a = ESIEEAuthor.query.filter(ESIEEAuthor.nom=='REDIS').first()
a.prenom 

'Jean'

In [112]:
Labo.query.all()

[<Lab esycom>,
 <Lab ligm>,
 <Lab LIGM>,
 <Lab ESYCOM>,
 <Lab LISIS>,
 <Lab IRG>,
 <Lab ESIEE>]

### Initialisation de la base avec les publis

In [113]:
Tab = pd.read_csv("csv/2016-17.csv", sep=",", header=0, escapechar="\\", error_bad_lines=False)
Tab 

,BibliographyType,ISBN,Identifier,Author,Title,Journal,Volume,Number,Month,Pages,...,Howpublished,Institution,Organizations,School,Annote,Custom1,Custom2,Custom3,Custom4,Custom5
0,7,NaN,allab:hal-01578585,"Allab, Yacine; Guo, Xiaofeng; Pellegrino, Marg...",Energy and comfort assessment in educational b...,{Energy and Buildings},143,NaN,NaN,202-219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,indoor air quality ; energy audit ; thermal co...,https://hal.archives-ouvertes.fr/hal-01578585/...,NaN
1,7,NaN,bahri:hal-01286949,"Bahri, Nejmeddine; Grandpierre, Thierry; Ayed,...",Embedded Real-Time H264/AVC High Definition Vi...,{Journal of Signal Processing Systems},86,1.0,NaN,67-84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Real-time ; Embedded system ; GOP level paral...,https://hal.archives-ouvertes.fr/hal-01286949/...,NaN
2,7,NaN,bahri:hal-01797084,"Bahri, Nejmeddine; Belhadj, Nidhameddine; Gran...",Real-time H264/AVC encoder based on enhanced f...,{Journal of Real-Time Image Processing},12,4.0,December,791 - 812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H264/AVC encoder ; DSP ; multi-core ; Frame Le...,https://hal.archives-ouvertes.fr/hal-01797084/...,NaN
3,7,NaN,bennour:hal-01708912,"Bennour, Alae; Moutier, Fr{'e}d{'e}ric; Polleu...",A Distributed Extended Ebers--Moll Model Topol...,{IEEE Transactions on Electron Devices},64,5.0,May,2267 - 2274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,NaN,bourdeau:hal-01648411,"Bourdeau, Mathieu; Guo, Xiaofeng & Nefzaoui, E.",Buildings energy consumption generation gap: a...,{Energy and Buildings},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,statistical analysis ; Energy Consumption ; En...,https://hal.archives-ouvertes.fr/hal-01648411/...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,10,NaN,jaffrezicrenault:hal-01579595,"Jaffrezic-Renault, Nicole & Lissorgues, Gaelle...",Sensors vol.17(8) Special Issue ''State-of-the...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mechanical sensors ; MEMS ; RF MEMS ; MOEMS ; ...,NaN,NaN
131,10,NaN,midelet:hal-01637925,"Midelet, Clyde; Le Pioufle, Bruno; Fran{c c}ai...",Analysis of dielectrophoretic trapping of func...,NaN,NaN,NaN,May,NaN,...,{Molecular Plasmonics 2017},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,14,NaN,dutta:hal-01360457,"Dutta, Kunal; Ghosh, Arijit & Mustafa, Nabil",A new asymmetric correlation inequality for Ga...,NaN,NaN,NaN,September,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Convex bodies ; Gaussian measure ; correlation...,https://hal.archives-ouvertes.fr/hal-01360457/...,NaN
133,14,NaN,hendel:hal-01375603,"Hendel, Martin; Azos-Diaz, Karina & Tremeac, B...",Behavioral Adaptation to Heat-Related Health R...,NaN,NaN,NaN,November,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,heat-waves ; heat-related mortality ; climate ...,https://hal.archives-ouvertes.fr/hal-01375603/...,NaN


In [114]:
Tab.columns

Index(['BibliographyType', 'ISBN', 'Identifier', 'Author', 'Title', 'Journal',
       'Volume', 'Number', 'Month', 'Pages', 'Year', 'Address', 'Note', 'URL',
       'Booktitle', 'Chapter', 'Edition', 'Series', 'Editor', 'Publisher',
       'ReportType', 'Howpublished', 'Institution', 'Organizations', 'School',
       'Annote', 'Custom1', 'Custom2', 'Custom3', 'Custom4', 'Custom5'],
      dtype='object')

In [115]:
authorsDict = {u.nom.lower():u for u in ESIEEAuthor.query.all()}

In [116]:
authorsDict.keys()

dict_keys(['myriam', 'bill', 'joe', 'kelly', 'bertrand', 'mainwaring', 'baudoin', 'de cambry', 'fakri', 'couprie', 'serrano', 'villegas', 'anderson', 'dhermies', 'georges', 'latorre', 'degny', 'kahane', 'reama', 'cela', 'lahille', 'natowicz', 'roudier', 'douay', 'sevely', 'mimoun', 'reille', 'venard', 'wirth', 'bureau', 'debbasch', 'lefebvre', 'schoen', 'eve', 'blanchard', 'courivaud', 'chu', 'bercher', 'delabie', 'perroton', 'pointet', 'poulichet', 'talbot', 'todman', 'berland', 'george', 'najman', 'bourouina', 'zampera', 'madaoui', 'redis', 'habib', 'lissorgues', 'francais', 'grandpierre', 'kocik', 'roy', 'exertier', 'basset', 'castro goncalves', 'gervais de lafond', 'valbin', 'hamouche', 'hilaire', 'abdeddaim', 'buzer', 'polleux', 'zitoune', 'dokladalova', 'algre', 'berraf', 'delemarle', 'mustafa', 'gardes', 'kachouri', 'noury', 'zangar', 'cousty', 'nadal', 'villard', 'ait saadi', 'alves', 'masson', 'ugon', 'wang', 'chierchia', 'guo', 'nefzaoui', 'perret', 'negrel', 'behiri', 'hende

In [117]:
"""from app import db
from app.models import ESIEEAuthor, Labos

import csv
import re"""

def splitAuthors(authors):
    l = authors.split(';')
    w = [x.split(',')[0].strip() for x in l]
    if '&' in l[-1]:
        w.append(l[-1].split('&')[1].split(',')[0].strip())
    return w

import pandas as pd


Tab = pd.read_csv("csv/2018-19.csv", sep=",", header=0, escapechar="\\", error_bad_lines=False)

listColumns = ['BibliographyType', 'ISBN', 'Identifier', 'Author', 'Title', 'Journal',
       'Volume', 'Number', 'Month', 'Pages', 'Year', 'Address', 'Note', 'URL',
       'Booktitle', 'Chapter', 'Edition', 'Series', 'Editor', 'Publisher',
       'ReportType', 'Howpublished', 'Institution', 'Organizations', 'School',
       'Annote', 'Custom1', 'Custom2', 'Custom3', 'Custom4', 'Custom5'],

for idx in Tab.index:
        
    if not Publis.query.filter(Publis.Identifier==Tab.loc[idx, 'Identifier']).first(): #Test si pas déjà dans la base       
        authors = splitAuthors(Tab.loc[idx, 'Author'])
        listAuthors = []
        for author in authors:
            if author.lower() in authorsDict.keys():
                listAuthors.append(authorsDict[author.lower()])

        publi = Publis()
            #id = Tab.loc[idx, 'Identifier']) 
        for attribut in dir(publi): 
            if not attribut.startswith('_') :
                try:
                    if not attribut=='Year': 
                        setattr(publi, attribut, Tab.loc[idx, attribut])
                except:
                    pass
        publi.Author = listAuthors 
        publi.Year = int(Tab.loc[idx,'Year'])
        publi.authors = Tab.loc[idx, 'Author']
        db.session.add(publi)
        db.session.commit()

### Exemples d'utilisation/interrogation

In [122]:
# Auteurs d'un labo
Labo.query.filter(Labo.labname=='LIGM').first().membres.filter().all()

[<Author BERTRAND>,
 <Author COUPRIE>,
 <Author CHU>,
 <Author BERCHER>,
 <Author TALBOT>,
 <Author GEORGE>,
 <Author NAJMAN>,
 <Author GRANDPIERRE>,
 <Author HILAIRE>,
 <Author ABDEDDAIM>,
 <Author BUZER>,
 <Author DOKLADALOVA>,
 <Author BERRAF>,
 <Author MUSTAFA>,
 <Author KACHOURI>,
 <Author ZANGAR>,
 <Author COUSTY>,
 <Author AIT SAADI>,
 <Author MASSON>,
 <Author WANG>,
 <Author CHIERCHIA>,
 <Author PERRET>,
 <Author NEGREL>,
 <Author SAHLI>]

In [123]:
# publi d'un auteur
ll = ESIEEAuthor.query.filter(ESIEEAuthor.nom=='COUPRIE').first().publis
ll

[<Publis robic:hal-02155490>, <Publis robic:hal-02169702>]

In [125]:
# toutes les pubs d'un labo
w = Publis.query.join(Publis.Author).join(ESIEEAuthor.labo).filter(Labo.labname=="LIGM").all() 

In [127]:
# Toutes les pubs d'un labo sur une période
pp = Publis.query.join(Publis.Author).join(ESIEEAuthor.labo).filter(Labo.labname=="LIGM")\
   .filter(Publis.Year<2018).filter(Publis.Year>2016).all() 
for x in pp: 
    print(x.Identifier, x.Year)

In [128]:
for attribut in dir(publi): 
    if not attribut.startswith('_') :
        try:
            setattr(publi, attribut, Tab.loc[idx, attribut])
        except:
            pass
publi.Author = listAuthors        

In [129]:
[a.labo for a in listAuthors]

[<Lab LIGM>, <Lab LIGM>, <Lab LIGM>]

### Génération d'un graphique


In [131]:
ListeAuthors = [(au.nom,au.nom) for au in ESIEEAuthor.query.all()]

In [132]:
from datetime import date
labo = "LIGM"
allPubs = Publis.query.join(Publis.Author).join(ESIEEAuthor.labo).filter(Labo.labname==labo).all() 
monthsDict = {'January': 1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 
              'September':9, 'October':10, 'November':11, 'December':12 , None:1}
listDates = [date(pub.Year, monthsDict[pub.Month], 1) for pub in allPubs]
dateIndex = pd.date_range('2018-01-01', periods=24, freq='MS').to_native_types()
tab = pd.DataFrame(index=dateIndex, columns=['vals'])
tab['vals'] = 0
for d in listDates:
    tab.loc[str(d), 'vals'] = tab.loc[str(d), 'vals']+1

pd.options.plotting.backend = "pandas_bokeh"
w = tab.plot(kind='bar',vertical_xlabel=True, return_html=True)


In [133]:
import matplotlib.pyplot as plt

In [134]:
import bokeh

In [135]:
pd.options.plotting.backend = "pandas_bokeh"

In [136]:
w = tab.plot(kind='bar',vertical_xlabel=True, return_html=True)

In [137]:
from IPython.display import HTML
HTML(w)

In [138]:
pd.plotting.output_notebook()

Loading BokehJS ...

In [139]:
str(listDates[2])

'2019-05-01'

In [140]:
tab

,vals
2018-01-01,0
2018-02-01,0
2018-03-01,0
2018-04-01,0
2018-05-01,0
2018-06-01,0
2018-07-01,0
2018-08-01,0
2018-09-01,4
2018-10-01,2


In [142]:
ListeAuthors.sort()

In [143]:
ListeAuthors

[('ABDEDDAIM', 'ABDEDDAIM'),
 ('AIT SAADI', 'AIT SAADI'),
 ('ALGRE', 'ALGRE'),
 ('ALVES', 'ALVES'),
 ('ANDERSON', 'ANDERSON'),
 ('BASSET', 'BASSET'),
 ('BAUDOIN', 'BAUDOIN'),
 ('BEHIRI', 'BEHIRI'),
 ('BERCHER', 'BERCHER'),
 ('BERLAND', 'BERLAND'),
 ('BERRAF', 'BERRAF'),
 ('BERTRAND', 'BERTRAND'),
 ('BLANCHARD', 'BLANCHARD'),
 ('BOUROUINA', 'BOUROUINA'),
 ('BUREAU', 'BUREAU'),
 ('BUZER', 'BUZER'),
 ('Bill', 'Bill'),
 ('CASTRO GONCALVES', 'CASTRO GONCALVES'),
 ('CELA', 'CELA'),
 ('CHIERCHIA', 'CHIERCHIA'),
 ('CHU', 'CHU'),
 ('COUPRIE', 'COUPRIE'),
 ('COURIVAUD', 'COURIVAUD'),
 ('COUSTY', 'COUSTY'),
 ('DE CAMBRY', 'DE CAMBRY'),
 ('DEBBASCH', 'DEBBASCH'),
 ('DEGNY', 'DEGNY'),
 ('DELABIE', 'DELABIE'),
 ('DELEMARLE', 'DELEMARLE'),
 ('DHERMIES', 'DHERMIES'),
 ('DOKLADALOVA', 'DOKLADALOVA'),
 ('DOUAY', 'DOUAY'),
 ('DRAIN', 'DRAIN'),
 ('EVE', 'EVE'),
 ('EXERTIER', 'EXERTIER'),
 ('FAKRI', 'FAKRI'),
 ('FRANCAIS', 'FRANCAIS'),
 ('GARDES', 'GARDES'),
 ('GEORGE', 'GEORGE'),
 ('GEORGES', 'GEORGES'),
